In [1]:
from pyspark.sql import SparkSession

In [2]:
# Initialize SparkSession
spark = (SparkSession.builder
        .appName("cs744")
        .master("spark://master:7077")
        .config("spark.driver.memory", "20g")
        .config("spark.executor.memory", "20g")
        .config("spark.executor.cores", "4")
        .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/01/30 19:24:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Reading Data

In [3]:
holidays = spark.read.csv("hdfs://nn:9000/holidays2.csv", inferSchema=True, header=True)

In [4]:
holidays.limit(3).toPandas()

,date,holiday
0,01/01/2013,New Year's Day
1,01/01/2014,New Year's Day
2,01/01/2015,New Year's Day


In [5]:
calls = spark.read.csv("hdfs://nn:9000/sf.csv", inferSchema=True, header=True)

In [6]:
calls.limit(3).toPandas()

25/01/30 19:25:19 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20


# Filtering

In [28]:
from pyspark.sql.functions import col

In [29]:
holidays_14 = holidays.filter(col("date").contains("2014"))

In [30]:
holidays_14_birth = holidays_1.filter(col("holiday").contains("Birthday"))

In [31]:
holidays_14_birth.count()

2

In [32]:
holidays_2.show()

+----------+--------------------+
|      date|             holiday|
+----------+--------------------+
|01/20/2014|Birthday of Marti...|
|02/17/2014|Washington's Birt...|
+----------+--------------------+



In [47]:
from pyspark.sql.functions import expr

In [58]:
holidays.filter(expr("holiday like '%Birthday%'")).count()

24

In [59]:
holidays.filter(expr("holiday like '%Birthday%' AND date like '%2014'")).count()

2

# Create New Column

In [33]:
from pyspark.sql.functions import lower

In [34]:
holidays_14 = holidays_14.withColumn("holiday_lower", lower(col("holiday")))

In [35]:
holidays_14

DataFrame[date: string, holiday: string, holiday_lower: string]

In [36]:
holidays_14_birth = holidays_14.filter(col("holiday_lower").contains("birthday"))

In [37]:
holidays_14_birth.count()

2

# Select

In [44]:
calls.select("Call Date")

DataFrame[Call Date: string]

In [45]:
calls.select("Call Date", "Call Type").limit(3).toPandas()

,Call Date,Call Type
0,05/01/2022,Outside Fire
1,01/19/2022,Alarms
2,05/03/2021,Alarms


# Grouping

In [68]:
# SELECT SUM(`Number of Alarms`) as "Total Alarms"
# FROM calls
# GROUP BY `Call Type`
# WHERE `Call Date` = '01/01/2014'

In [70]:
calls_date = calls.filter(expr("`Call Date` = '01/01/2014'"))

In [71]:
calls_date.limit(3).toPandas()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,140010567,E36,14000479,Structure Fire,01/01/2014,01/01/2014,01/01/2014 09:32:54 PM,01/01/2014 09:33:31 PM,01/01/2014 09:36:27 PM,01/01/2014 09:37:00 PM,...,Fire,1,ENGINE,12,4,5,Western Addition,140010567-E36,POINT (-122.427815582933 37.784448089516),39
1,140010061,KM12,14000046,Structure Fire,01/01/2014,12/31/2013,01/01/2014 12:43:09 AM,01/01/2014 12:44:07 AM,01/01/2014 12:47:31 AM,01/01/2014 12:47:31 AM,...,Alarm,1,PRIVATE,7,4,2,Marina,140010061-KM12,POINT (-122.429500504516 37.801174806033),13
2,140010523,B08,14000439,Alarms,01/01/2014,01/01/2014,01/01/2014 06:47:53 PM,01/01/2014 06:50:10 PM,01/01/2014 06:50:17 PM,01/01/2014 06:50:59 PM,...,Alarm,1,CHIEF,2,8,7,Inner Sunset,140010523-B08,POINT (-122.47140400613 37.761010743685),14


In [74]:
from pyspark.sql.functions import sum

In [77]:
calls_grouped = calls_date.groupBy("Call Type").agg(sum("Number of Alarms").alias("Num_Alarms"))

In [78]:
calls_grouped.show()

[Stage 60:====================================================>   (16 + 1) / 17]

+--------------------+----------+
|           Call Type|Num_Alarms|
+--------------------+----------+
|              Alarms|        82|
|      Structure Fire|       164|
|Elevator / Escala...|         1|
|Citizen Assist / ...|        10|
|        Vehicle Fire|         2|
|               Other|        25|
|        Outside Fire|        35|
|   Traffic Collision|        54|
|Gas Leak (Natural...|         7|
|        Water Rescue|        13|
|    Medical Incident|       818|
|          Fuel Spill|         5|
|Smoke Investigati...|         2|
+--------------------+----------+



# Caching

In [96]:
calls_high = calls_grouped.filter(expr("Num_Alarms > 100"))

In [97]:
%%time
calls_high.count()

[Stage 98:=================================================>      (15 + 2) / 17]

CPU times: user 8.06 ms, sys: 17.4 ms, total: 25.5 ms
Wall time: 4.58 s


2

In [98]:
calls_grouped.cache()

DataFrame[Call Type: string, Num_Alarms: bigint]

In [99]:
%%time
calls_high.count()

[Stage 104:===================================================>   (16 + 1) / 17]

CPU times: user 16.6 ms, sys: 13.8 ms, total: 30.4 ms
Wall time: 4.95 s


2

In [100]:
%%time
calls_high.count()

CPU times: user 2.63 ms, sys: 486 μs, total: 3.11 ms
Wall time: 426 ms


2

In [101]:
calls_grouped.unpersist()

DataFrame[Call Type: string, Num_Alarms: bigint]

# Join

# Partition

In [55]:
calls_14 = calls.filter(expr("`Call Date` like '%2014'"))
calls_14.count()

280905

In [56]:
calls.rdd.getNumPartitions()

17